# Welcome to our Logistic Regression Workshop!

In our previous workshop, we discussed what linear regression is, and how it mathematically looks like. Now, we are going to start learning a more complicated regression algorithm, _logistic regression_. We will start by describing what logistic regression is, then give some mathematical intuition of how it works. Then, similar to previous week, we will implement our own logistic regression model and compare its performance to our linear regression model.


Let's start by understanding what _logistic regression_ is.

### Logistic Regression

- Logistic regression functions are of the form `Y = sigmoid(wX + b)`, where

  `Y`: dependent variable vector, the probability that an instance belongs to a particular class

  `w`: weight vector, the importance of each independent variable. We will refer to weights as `coefficients` in this workshop, to make the code more understandable.

  `X`: independent variable vector, what we base our predictions on

  `b`: bias, helps with fitting the model better. We will refer to bias as `intercept` in this workshop, again to make the code more understandable.

- The purpose of `finding` the most accurate model requires us to find the best parameters `w` and `b` such that our logistic function fits our dataset well. That is, we try many many many different `w` and `b` values to find the logistic function that helps us the most. But how do we find them?
